In [1]:
import torch
from transformers import AutoProcessor,Qwen2VLForConditionalGeneration
from PIL import Image
from pdf2image import convert_from_path
from PIL import Image
import os
from tqdm import tqdm
import json
from jiwer import wer, cer, process_words

import re

/Users/maryamsaad/Documents/arabic-ocr-transformer/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# load documents and convert them to images

In [2]:


def pdf_to_images(path, dpi=300):
    images = []
    if os.path.isfile(path):
        images.extend(convert_from_path(path, dpi=dpi))
    else:
        for pdf_file in tqdm(os.listdir(path), desc="Converting PDFs"):
            if pdf_file.lower().endswith('.pdf'):
                pdf_path = os.path.join(path, pdf_file)
                images.extend(convert_from_path(pdf_path, dpi=dpi))
    print(f"\n✅ Conversion complete!")
    return images

pages = pdf_to_images("/Users/maryamsaad/Documents/grad_data/unit1.pdf")


✅ Conversion complete!


In [3]:
len(pages)  

10

# OCR process

In [4]:

model_id = "NAMAA-Space/Qari-OCR-v0.3-VL-2B-Instruct"
processor_id = "Qwen/Qwen2-VL-2B-Instruct"

# Use CPU for now (MPS has memory issues with large vision models)
device = torch.device("mps")
print(device)

# Load model with float32 for CPU
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16, 
    trust_remote_code=True,
    cache_dir="./Qari_model_cache"
).to(device)

# Load processor
processor = AutoProcessor.from_pretrained(
    processor_id,
    trust_remote_code=True,
    cache_dir="./Qwen_processor_cache"
)

print(f"✅ Model ready on {device}")


`torch_dtype` is deprecated! Use `dtype` instead!


mps


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


✅ Model ready on mps


In [5]:
def get_ocr_prompt() -> str:
    """Get the OCR instruction prompt for Arabic text."""
    return """
        You are an Arabic OCR specialist designed to extract text from images containing Arabic content. When processing images:
        1. ALWAYS read Arabic text from right to left, and top to bottom.
        3. DO NOT hallucinate text when encountering:
           - Decorative elements
           - Page numbers, stamps, or margin notes
           - Blurry or unclear text
        4. When text is unclear, mark it as [غير واضح].
        5. Only transcribe visible text — accuracy > completeness.
        6. Ignore watermarks and non-text background elements.
        8. Preserve original formatting, including line breaks and spacing.
        9. Do not hallucinate punctuation or diacritics. 
        10. Output the extracted text as it is visually presented in the image.
    """


In [6]:
def extract_text_from_image(images, model, processor, max_size=(1024, 1024)):
    ocr_results = []

    for img in tqdm(images, desc="Extracting text"):
        # Resize image if too large
        if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
            img.thumbnail(max_size, Image.Resampling.LANCZOS)
        
        prompt_messages = [
            {"role": "system", "content": get_ocr_prompt()},
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": img},
                    {"type": "text", "text": "Extract text from the provided image accurately."}
                ]
            }
        ]
        # Apply chat template
        prompt_text = processor.apply_chat_template(
            prompt_messages,
            add_generation_prompt=True,
            tokenize=False
        )

        # Tokenize with image input
        inputs = processor(
            text=prompt_text,
            images=img,
            return_tensors="pt",
            padding=True
        ).to(model.device)

        # Run inference with valid parameters only
        with torch.inference_mode():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=1500,
                do_sample=False, 
                repetition_penalty=1.4,
                # pad_token_id=processor.tokenizer.eos_token_id
            )
        

        generated_text = processor.batch_decode(
            generated_ids, 
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]
        ocr_results.append(generated_text.strip())

    return ocr_results

In [7]:
pages[:3]

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2480x3509>]

In [8]:
new_test= extract_text_from_image(pages, model, processor)

Extracting text:   0%|          | 0/10 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Extracting text: 100%|██████████| 10/10 [09:39<00:00, 57.90s/it]


In [10]:
extracted_text=new_test

In [11]:
extracted_text

['system\n\n        You are an Arabic OCR specialist designed to extract text from images containing Arabic content. When processing images:\n        1. ALWAYS read Arabic text from right to left, and top to bottom.\n        3. DO NOT hallucinate text when encountering:\n           - Decorative elements\n           - Page numbers, stamps, or margin notes\n           - Blurry or unclear text\n        4. When text is unclear, mark it as [غير واضح].\n        5. Only transcribe visible text — accuracy > completeness.\n        6. Ignore watermarks and non-text background elements.\n        8. Preserve original formatting, including line breaks and spacing.\n        9. Do not hallucinate punctuation or diacritics. \n        10. Output the extracted text as it is visually presented in the image.\n    \nuser\nExtract text from the provided image accurately.\nassistant\n<p>كسب <i>الأصدقاء</i> بسهولة، حب مشاركة الآخرين في أفراغهم وأحزانهنم ، نقبل الآخرن، بدء الحوار مع الآخرn، إعطاء النصائح للأسد

# Page reconstruction

In [12]:
from bs4 import BeautifulSoup
def reconstruct_pages(html_content):
    reconstructed_pages = []
    for content in html_content:
        soup = BeautifulSoup(content, "html.parser")
        plain_text = soup.get_text(separator="\n")
        reconstructed_pages.append(plain_text)
    return reconstructed_pages

reconstructed_texts = reconstruct_pages(extracted_text)

In [19]:
reconstructed_texts

['system\n\n        You are an Arabic OCR specialist designed to extract text from images containing Arabic content. When processing images:\n        1. ALWAYS read Arabic text from right to left, and top to bottom.\n        3. DO NOT hallucinate text when encountering:\n           - Decorative elements\n           - Page numbers, stamps, or margin notes\n           - Blurry or unclear text\n        4. When text is unclear, mark it as [غير واضح].\n        5. Only transcribe visible text — accuracy > completeness.\n        6. Ignore watermarks and non-text background elements.\n        8. Preserve original formatting, including line breaks and spacing.\n        9. Do not hallucinate punctuation or diacritics. \n        10. Output the extracted text as it is visually presented in the image.\n    \nuser\nExtract text from the provided image accurately.\nassistant\n\nكسب \nالأصدقاء\n بسهولة، حب مشاركة الآخرين في أفراغهم وأحزانهنم ، نقبل الآخرن، بدء الحوار مع الآخرn، إعطاء النصائح للأسدفاةn

In [18]:
for i,page_text in enumerate(reconstructed_texts):
    reconstructed_texts[i]=page_text.replace('\n','').strip()
    reconstructed_texts[i]=page_text.replace('system\n\n        You are an Arabic OCR specialist designed to extract text from images containing Arabic content. When processing images:\n        1. ALWAYS read Arabic text from right to left, and top to bottom.\n        3. DO NOT hallucinate text when encountering:\n           - Decorative elements\n           - Page numbers, stamps, or margin notes\n           - Blurry or unclear text\n        4. When text is unclear, mark it as [غير واضح].\n        5. Only transcribe visible text — accuracy > completeness.\n        6. Ignore watermarks and non-text background elements.\n        8. Preserve original formatting, including line breaks and spacing.\n        9. Do not hallucinate punctuation or diacritics. \n        10. Output the extracted text as it is visually presented in the image.\n    \nuser\nExtract text from the provided image accurately.\nassistant',' ')
    reconstructed_texts[i]=page_text.replace('system\n\n        You are an Arabic OCR specialist designed to extract text from images containing Arabic content. When processing images:\n        1. ALWAYS read Arabic text from right to left, and top to bottom.\n        2. For poetry or columned text, process the rightmost section first, preserving the format.\n        3. DO NOT hallucinate text when encountering:\n           - Decorative elements\n           - Page numbers, stamps, or margin notes\n           - Blurry or unclear text\n        4. When text is unclear, mark it as [غير واضح].\n        5. Only transcribe visible text — accuracy > completeness.\n        6. Ignore watermarks and non-text background elements.\n    \nuser\nExtract text from the provided image accurately.\nassistant\n\n',' ')

# Evaluate

In [226]:
gt_paths="/Users/maryamsaad/Documents/grad_data/ground_truth_files/unit1ground_truth_clean.json"
with open(gt_paths, 'r', encoding='utf-8') as f:
    ground_truth_data = json.load(f)
ground_truth_data

{'1': 'كسب الأصدقاء بسهولة، حب مشاركة الآخرين في أفراحهم وأحزانهم، تق بل الآخر، بدء الحوار مع \nالآخر، إ عطاء النصائح للأصدقاء، والقدرة على القيادة  ويتضح هذا النوع من الذكاء أكثر لدى القادة \n،و رجال الدين ، والأخصائيين النفسيين، والأخصائيين الاجتماعيين والمعالجين. \nلذا يمكن أن تنمي ذكاءك الاجتماعي من خلال : - \n1-ممارسة الرياضات الجماعية \n2-المشاركة في جماعاتالنشاط المدرسية. \n3-.يرود لكشب براقلأاو ءاقدصلأا ةرايز \n4-إدارة الحوا ر مع الآخرين. \n5-تعليم اخرين . \n6- القيام بأدوار قيادية \nتطبيق : نظم لرحلة مع الاخصائي الإجتماعي يمكن أن تقوم بها المدرسة ، موضحا فقرات ا لأنشطة \nومن الذي سيدير كل نشاط من هذه الأنشطة. \nسابعا الذكاء الشخصي  : Personal Intelligence \n  يشير إلىقدرة الفرد على فهم ذاته و   امكاناته الحقيقية وتمييزها وتصنيفها، والتصرف تبعا لهذه \nالإمكانات.  \n  ويتضمن هذا الذكاء معرفة الفرد لنفسه ولقدراته الحقيقية، ومعرفته كذلك لجوانب ضعفه وقوته . \nلذا علينا نحن الآباء والمعلمين أن نتقبل هذا وألا نطلب من أبنائنا وتلاميذنا أكثر مما تسمح به \nإلا دق حومط ىوتسم مهل عضن لاو 

In [227]:
for page_text in ground_truth_data:
    ground_truth_data[page_text] = ground_truth_data[page_text].replace('\n', '').strip()
    ground_truth_data[page_text] = re.sub(r'[^\w\s\u0600-\u06FF.,؛،؟!-]', ' ', ground_truth_data[page_text])
print(ground_truth_data)

{'1': 'كسب الأصدقاء بسهولة، حب مشاركة الآخرين في أفراحهم وأحزانهم، تق بل الآخر، بدء الحوار مع الآخر، إ عطاء النصائح للأصدقاء، والقدرة على القيادة  ويتضح هذا النوع من الذكاء أكثر لدى القادة ،و رجال الدين ، والأخصائيين النفسيين، والأخصائيين الاجتماعيين والمعالجين. لذا يمكن أن تنمي ذكاءك الاجتماعي من خلال   - 1-ممارسة الرياضات الجماعية 2-المشاركة في جماعاتالنشاط المدرسية. 3-.يرود لكشب براقلأاو ءاقدصلأا ةرايز 4-إدارة الحوا ر مع الآخرين. 5-تعليم اخرين . 6- القيام بأدوار قيادية تطبيق   نظم لرحلة مع الاخصائي الإجتماعي يمكن أن تقوم بها المدرسة ، موضحا فقرات ا لأنشطة ومن الذي سيدير كل نشاط من هذه الأنشطة. سابعا الذكاء الشخصي    Personal Intelligence   يشير إلىقدرة الفرد على فهم ذاته و   امكاناته الحقيقية وتمييزها وتصنيفها، والتصرف تبعا لهذه الإمكانات.    ويتضمن هذا الذكاء معرفة الفرد لنفسه ولقدراته الحقيقية، ومعرفته كذلك لجوانب ضعفه وقوته . لذا علينا نحن الآباء والمعلمين أن نتقبل هذا وألا نطلب من أبنائنا وتلاميذنا أكثر مما تسمح به إلا دق حومط ىوتسم مهل عضن لاو ، مهتاناكم تمكنهم قدراتهم من الوصو

In [ ]:
cleaned_gt = reconstruct_pages(list(ground_truth_data.values()))

In [229]:
cleaned_gt

['كسب الأصدقاء بسهولة، حب مشاركة الآخرين في أفراحهم وأحزانهم، تق بل الآخر، بدء الحوار مع الآخر، إ عطاء النصائح للأصدقاء، والقدرة على القيادة  ويتضح هذا النوع من الذكاء أكثر لدى القادة ،و رجال الدين ، والأخصائيين النفسيين، والأخصائيين الاجتماعيين والمعالجين. لذا يمكن أن تنمي ذكاءك الاجتماعي من خلال   - 1-ممارسة الرياضات الجماعية 2-المشاركة في جماعاتالنشاط المدرسية. 3-.يرود لكشب براقلأاو ءاقدصلأا ةرايز 4-إدارة الحوا ر مع الآخرين. 5-تعليم اخرين . 6- القيام بأدوار قيادية تطبيق   نظم لرحلة مع الاخصائي الإجتماعي يمكن أن تقوم بها المدرسة ، موضحا فقرات ا لأنشطة ومن الذي سيدير كل نشاط من هذه الأنشطة. سابعا الذكاء الشخصي    Personal Intelligence   يشير إلىقدرة الفرد على فهم ذاته و   امكاناته الحقيقية وتمييزها وتصنيفها، والتصرف تبعا لهذه الإمكانات.    ويتضمن هذا الذكاء معرفة الفرد لنفسه ولقدراته الحقيقية، ومعرفته كذلك لجوانب ضعفه وقوته . لذا علينا نحن الآباء والمعلمين أن نتقبل هذا وألا نطلب من أبنائنا وتلاميذنا أكثر مما تسمح به إلا دق حومط ىوتسم مهل عضن لاو ، مهتاناكم تمكنهم قدراتهم من الوصول الي

# extracting errors

In [230]:
import re
import arabic_reshaper


def normalize_arabic(text):
    text = re.sub(r'[\u064B-\u0652]', '', text)
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'[يى]', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ـ', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

gt_normalized = [normalize_arabic(text) for text in cleaned_gt]
reconstructed_normalized = [normalize_arabic(text) for text in reconstructed_texts]

In [231]:
cleaned_gt

['كسب الأصدقاء بسهولة، حب مشاركة الآخرين في أفراحهم وأحزانهم، تق بل الآخر، بدء الحوار مع الآخر، إ عطاء النصائح للأصدقاء، والقدرة على القيادة  ويتضح هذا النوع من الذكاء أكثر لدى القادة ،و رجال الدين ، والأخصائيين النفسيين، والأخصائيين الاجتماعيين والمعالجين. لذا يمكن أن تنمي ذكاءك الاجتماعي من خلال   - 1-ممارسة الرياضات الجماعية 2-المشاركة في جماعاتالنشاط المدرسية. 3-.يرود لكشب براقلأاو ءاقدصلأا ةرايز 4-إدارة الحوا ر مع الآخرين. 5-تعليم اخرين . 6- القيام بأدوار قيادية تطبيق   نظم لرحلة مع الاخصائي الإجتماعي يمكن أن تقوم بها المدرسة ، موضحا فقرات ا لأنشطة ومن الذي سيدير كل نشاط من هذه الأنشطة. سابعا الذكاء الشخصي    Personal Intelligence   يشير إلىقدرة الفرد على فهم ذاته و   امكاناته الحقيقية وتمييزها وتصنيفها، والتصرف تبعا لهذه الإمكانات.    ويتضمن هذا الذكاء معرفة الفرد لنفسه ولقدراته الحقيقية، ومعرفته كذلك لجوانب ضعفه وقوته . لذا علينا نحن الآباء والمعلمين أن نتقبل هذا وألا نطلب من أبنائنا وتلاميذنا أكثر مما تسمح به إلا دق حومط ىوتسم مهل عضن لاو ، مهتاناكم تمكنهم قدراتهم من الوصول الي

In [232]:
def extract_errors(ground_truth_data, reconstructed_texts):
    errors = []
    mean_wer = 0
    mean_cer = 0
    for i, page_text in enumerate(reconstructed_texts):
        gt_text = ground_truth_data[i]
        wer_score = wer(gt_text, page_text) 
        cer_score = cer(gt_text, page_text) 
        errors.append({
            "page": i + 1,
            "wer": round(wer_score*100, 2),
            "cer": round(cer_score*100, 2)
        })
        mean_wer += wer_score
        mean_cer += cer_score
    mean_wer /= len(reconstructed_texts)
    mean_cer /= len(reconstructed_texts)
    errors.append({
        "page": "mean",
        "wer": round(mean_wer*100, 2),
        "cer": round(mean_cer*100, 2)
    })
    return errors

In [233]:
results = extract_errors(gt_normalized, reconstructed_normalized)
for r in results:
    print(f"Page {r['page']}: WER={r['wer']}%, CER={r['cer']}%")

Page 1: WER=44.01%, CER=16.39%
Page 2: WER=35.14%, CER=19.95%
Page 3: WER=43.9%, CER=22.64%
Page 4: WER=44.48%, CER=24.68%
Page 5: WER=32.37%, CER=12.09%
Page 6: WER=36.57%, CER=19.27%
Page 7: WER=41.08%, CER=21.34%
Page 8: WER=42.47%, CER=18.91%
Page 9: WER=51.39%, CER=30.43%
Page 10: WER=44.33%, CER=20.06%
Page mean: WER=41.57%, CER=20.58%


In [188]:
results = extract_errors(gt_normalized, reconstructed_normalized)
for r in results:
    print(f"Page {r['page']}: WER={r['wer']}%, CER={r['cer']}%")

Page 1: WER=42.61%, CER=15.67%
Page 2: WER=35.14%, CER=19.95%
Page 3: WER=43.9%, CER=23.28%
Page 4: WER=44.48%, CER=24.68%
Page 5: WER=32.66%, CER=14.43%
Page 6: WER=37.43%, CER=19.68%
Page 7: WER=41.08%, CER=21.34%
Page 8: WER=42.47%, CER=18.91%
Page 9: WER=51.39%, CER=30.43%
Page 10: WER=44.33%, CER=20.06%
Page mean: WER=41.55%, CER=20.84%


In [218]:
gt_normalized

['كسب الاصدقاء بسهوله، حب مشاركه الاخرين في افراحهم واحزانهم، تق بل الاخر، بدء الحوار مع الاخر، ا عطاء النصائح للاصدقاء، والقدره علي القياده ويتضح هذا النوع من الذكاء اكثر لدي القاده ،و رجال الدين ، والاخصائيين النفسيين، والاخصائيين الاجتماعيين والمعالجين. لذا يمكن ان تنمي ذكاءك الاجتماعي من خلال - 1-ممارسه الرياضات الجماعيه 2-المشاركه في جماعاتالنشاط المدرسيه. 3-.يرود لكشب براقلااو ءاقدصلاا هرايز 4-اداره الحوا ر مع الاخرين. 5-تعليم اخرين . 6- القيام بادوار قياديه تطبيق نظم لرحله مع الاخصائي الاجتماعي يمكن ان تقوم بها المدرسه ، موضحا فقرات ا لانشطه ومن الذي سيدير كل نشاط من هذه الانشطه. سابعا الذكاء الشخصي Personal Intelligence يشير اليقدره الفرد علي فهم ذاته و امكاناته الحقيقيه وتمييزها وتصنيفها، والتصرف تبعا لهذه الامكانات. ويتضمن هذا الذكاء معرفه الفرد لنفسه ولقدراته الحقيقيه، ومعرفته كذلك لجوانب ضعفه وقوته . لذا علينا نحن الاباء والمعلمين ان نتقبل هذا والا نطلب من ابنائنا وتلاميذنا اكثر مما تسمح به الا دق حومط يوتسم مهل عضن لاو ، مهتاناكم تمكنهم قدراتهم من الوصول اليه ، لال نوضرعتي

In [219]:
reconstructed_normalized

['system You are an Arabic OCR specialist designed to extract text from images containing Arabic content. When processing images: 1. ALWAYS read Arabic text from right to left, and top to bottom. 2. For poetry or columned text, process the rightmost section first, preserving the format. 3. DO NOT hallucinate text when encountering: - Decorative elements - Page numbers, stamps, or margin notes - Blurry or unclear text 4. When text is unclear, mark it as [غير واضح]. 5. Only transcribe visible text — accuracy > completeness. 6. Ignore watermarks and non-text background elements. user Extract text from the provided image accurately. assistant كسب الاصدقاء بسهوله، حب مشاركه الاخرين في افراغهم واحزانهم، نقبل الاخر، بدء الحوار مع الاخر، اعطاء النصائح للاصدقاء، والقدره علي القياده ويوضح هذا النوع من الذكاء اكثر لدي القاده ،ورجال الذين ، والاخصائيين النفسيين، والاخصائيين الاجتماعي والمعالجين. لذا يمكن ان تنمي ذكاء ك الاجتماعي من خلال : – -ممارسه الرياضات المدرسيه. ٢- المشاركه في جماعات النشاط ا